# User Experience Analytics



In [36]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
os.chdir('..')
from Db_connection.connection import PostgresConnection
from src.utils import *
from sklearn.cluster import KMeans
import seaborn as sns


In [3]:
db = PostgresConnection(dbname='telecom', user='postgres', password='postgres')
db.connect()

# Query the table to verify the write
query = "SELECT * FROM xdr_data_cleaned"
result = db.execute_query(query)

# Convert result to a DataFrame and display the information
df_cleaned = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df_cleaned.head(5))

# Close the connection
db.close_connection()

Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

## Aggregate metrics per customer

In [10]:
# Handle outliers
for col in ['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']:
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_cleaned[col] = np.where(df_cleaned[col] < lower_bound, df_cleaned[col].mean(), df_cleaned[col])
    df_cleaned[col] = np.where(df_cleaned[col] > upper_bound, df_cleaned[col].mean(), df_cleaned[col])

# Aggregate per customer
agg_df = df_cleaned.groupby('IMSI').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]
}).reset_index()

# Calculate total TCP retransmission, RTT, and throughput
agg_df['total_tcp'] = (agg_df['TCP DL Retrans. Vol (Bytes)'] + agg_df['TCP UL Retrans. Vol (Bytes)'])
agg_df['total_rtt'] = (agg_df['Avg RTT DL (ms)'] + agg_df['Avg RTT UL (ms)'])
agg_df['total_throughput'] = (agg_df['Avg Bearer TP DL (kbps)'] + agg_df['Avg Bearer TP UL (kbps)'])

# Drop the intermediary columns
agg_df = agg_df.drop(columns=[
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)'
])

agg_df.head()


,IMSI,Handset Type,total_tcp,total_rtt,total_throughput
0,2.040471e+14,Quectel Wireless. Quectel Ec21-E,2.156957e+07,127.458589,2.0
1,2.040808e+14,Quectel Wireless. Quectel Ec25-E,2.156957e+07,127.458589,1.0
2,2.082001e+14,Dn Electronics Danew Konnect 350,7.623557e+05,124.516329,109.0
3,2.082001e+14,Samsung Galaxy Grand (Gt-I9060X),2.797900e+04,84.000000,754.0
4,2.082001e+14,Apple iPhone 6S (A1688),2.156957e+07,92.000000,80.5


In [22]:
def compute_top_bottom_frequent(df_cleaned, column, top_n=10):
    return {
        'top': df_cleaned[column].nlargest(top_n).reset_index(drop=True),
        'bottom': df_cleaned[column].nsmallest(top_n).reset_index(drop=True),
        'most_frequent': df_cleaned[column].mode().reset_index(drop=True)
    }

tcp_stats = compute_top_bottom_frequent(agg_df, 'total_tcp')
rtt_stats = compute_top_bottom_frequent(agg_df, 'total_rtt')
throughput_stats = compute_top_bottom_frequent(agg_df, 'total_throughput')


        
print("Total TCP Stats", tcp_stats)
print("Total RTT Stats", rtt_stats)
print("Total Throughput Stats", throughput_stats)



Total TCP Stats {'top': 0    5.060485e+07
1    5.031799e+07
2    5.025940e+07
3    5.017006e+07
4    5.013602e+07
5    5.013342e+07
6    5.004143e+07
7    4.984284e+07
8    4.983777e+07
9    4.976102e+07
Name: total_tcp, dtype: float64, 'bottom': 0     97.0
1    128.0
2    129.0
3    134.0
4    143.0
5    176.0
6    176.0
7    177.0
8    179.0
9    182.0
Name: total_tcp, dtype: float64, 'most_frequent': 0    2.156957e+07
Name: total_tcp, dtype: float64}
Total RTT Stats {'top': 0    247.0
1    246.0
2    243.0
3    242.0
4    241.0
5    241.0
6    241.0
7    241.0
8    241.0
9    240.0
Name: total_rtt, dtype: float64, 'bottom': 0     0.0
1     0.0
2     2.0
3     4.0
4     5.0
5     6.0
6     8.0
7     9.0
8     9.0
9    10.0
Name: total_rtt, dtype: float64, 'most_frequent': 0    127.458589
Name: total_rtt, dtype: float64}
Total Throughput Stats {'top': 0    24306.0
1    24053.0
2    24005.0
3    23980.0
4    23960.0
5    23953.0
6    23951.0
7    23916.0
8    23886.0
9    23853.0
Name:

In [34]:
def report_distribution(df, group_col, value_col):
    return df.groupby(group_col)[value_col].mean()

throughput_distribution = report_distribution(agg_df, 'Handset Type', 'total_throughput')
tcp_distribution = report_distribution(agg_df, 'Handset Type', 'total_tcp')

print("\nAverage Throughput per Handset Type:\n")
print(throughput_distribution)
print("\nAverage TCP Retransmission per Handset Type:\n")
print(tcp_distribution)



Average Throughput per Handset Type:

Handset Type
A-Link Telecom I. Cubot A5                                                              5774.566744
A-Link Telecom I. Cubot Note Plus                                                       6699.000000
A-Link Telecom I. Cubot Note S                                                          8937.000000
A-Link Telecom I. Cubot Nova                                                           13836.171390
A-Link Telecom I. Cubot Power                                                          13628.171390
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809       58.000000
Zyxel Communicat. Lte7460                                                               9683.662864
Zyxel Communicat. Sbg3600                                                              15056.957888
Zyxel Communicat. Zyxel Wah7706                 

In [33]:
# Select features
features = agg_df[['total_tcp', 'total_rtt', 'total_throughput']]

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(features)
agg_df['cluster'] = kmeans.labels_

numeric_cols = agg_df.select_dtypes(include='number').columns


# Describe each cluster with only numeric columns
cluster_description = agg_df.groupby('cluster')[numeric_cols].mean()
print("\nCluster Descriptions:")
print(cluster_description)



Cluster Descriptions:
                 IMSI     total_tcp  total_rtt  total_throughput  cluster
cluster                                                                  
0        2.082015e+14  2.169949e+07  74.602957       1050.052893      0.0
1        2.082016e+14  1.214642e+06  84.977166       9335.687917      1.0
2        2.082016e+14  1.146207e+07  76.092598       6573.417498      2.0
